In [1]:
import gymnasium as gym


#定义环境
class MyWrapper(gym.Wrapper):

    def __init__(self):
        env = gym.make('CartPole-v1')
        super().__init__(env)
        self.env = env

    def reset(self, seed=None, options=None):
        state, info = self.env.reset()
        return state, info

    def step(self, action):
        state, reward, done, truncated, info = self.env.step(action)
        return state, reward, done, truncated, info


MyWrapper().reset()

(array([-0.00879188,  0.04424797, -0.02449973,  0.02518002], dtype=float32),
 {})

In [10]:
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
import time


def test_multiple_env(dumm, N):
    if dumm:
        #DummyVecEnv,在单线程中运行多个环境
        env = DummyVecEnv([MyWrapper] * N)
    else:
        #SubprocVecEnv,在多线程中运行多个环境
        env = SubprocVecEnv([MyWrapper] * N)

    start = time.time()

    #训练一个模型
    model = PPO('MlpPolicy', env, verbose=0, device='cpu').learn(total_timesteps=5000)

    print('消耗时间=', time.time() - start)

    #关闭环境
    env.close()

    #测试
    return evaluate_policy(model, MyWrapper(), n_eval_episodes=20)


test_multiple_env(dumm=True, N=2)

消耗时间= 6.790776014328003


(np.float64(351.2), np.float64(173.5057924105129))

In [4]:
test_multiple_env(dumm=True, N=10)

消耗时间= 11.926033973693848


(np.float64(403.5), np.float64(116.67583297324258))

In [11]:
test_multiple_env(dumm=False, N=2)

消耗时间= 8.359851121902466


(np.float64(334.0), np.float64(150.26875922825744))

In [12]:
test_multiple_env(dumm=False, N=10)

消耗时间= 12.964670896530151


(np.float64(321.75), np.float64(105.51960718274117))